In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
#from ipynb.fs.full.ToolBox import connect_to_db

In [ ]:
engine = connect_to_db()

In [2]:
# Returns a dataframe of the longest modec (flight level) == cfl (cleared flight level) plateau, filters out null values
def isolate_plateau(ifpsid):
    df = pd.read_sql_query("""SELECT instant,tas,plateau_cfl,cflok FROM sbx1.<table> WHERE ifpsid = '{ii}'""".format(ii=ifpsid),engine)
    df = df[df.tas.notna() & df.cflok == 1]
    if df.empty:
        return None
    df = df.loc[df.plateau_cfl == df.groupby('plateau_cfl').instant.apply(np.ptp).idxmax()]
    return df

# Applies a median filter to tas (true air speed) and picks out every nth value
def medfilt(df,ks=5):
    return signal.medfilt(df.tas,kernel_size=ks)[2::5]

In [ ]:
df_ifpsid = pd.read_sql_query(""" SELECT UNIQUE ifpsid FROM sbx1.<table>""" , engine)

In [ ]:
df_ifpsid.head()

In [ ]:
%%time
## filtering of suitable ifpsid
# Note: Will not work without database access. Use df_ifpsid_suitable.pickle for further processing.
d_list = []
for index, row in df_ifpsid.iterrows():
    df = pd.read_sql_query("""SELECT instant,tas,plateau_cfl,cflok FROM sbx1.<table> WHERE ifpsid = '{ii}'""".format(ii=row.ifpsid),engine)
    df_iso = isolate_plateau(row.ifpsid)
    if df_iso is None:
        continue
    med = medfilt(df_iso)
    if med.size < 30: #less than approx. 10 minutes
        continue
    d_data_i = {'ifpsid': row.ifpsid}
    d_list.append(d_data_i)
df_ifpsid_suitable = pd.DataFrame.from_dict(d_list)

In [3]:
df_ifpsid_suitable = pd.read_pickle("../pickles/df_ifpsid_suitable.pickle")

In [4]:
df_ifpsid_suitable.sample(10)

,ifpsid
65605,AA17217257
91290,AA15126585
83237,AA16302258
91928,AA15417655
73263,AA15313834
15646,AA16336089
139988,AA15194514
35960,AA16420459
174537,AA16479870
94907,AA15721022
